#The Modern Mamak: Estimating Provision Shops that were Live at the Time of Each Decade

Business directories were used to gather shop data and estimate when businesses ceased operations. However, these directories do not provide the actual termination date of a business. Instead, they record the most recent update based on ACRA data, reflecting the last time the business' status was checked.

For example, if a business is marked as *'Terminated as of 2017'*, it means that in 2017, the directory last confirmed the business was no longer active. Since directories generally stop updating a business' profile after confirming its closure, the actual shutdown may have occurred earlier. The last recorded updates in SGBusiness are mostly from 2017, while RecordOwl's last updates date back to 2008.

To estimate termination dates more accurately, data from both sources will be compared. The earliest recorded update is likely the closest to the true closure date, as later updates only confirm an already inactive status rather than providing new closure information.

<br>

*Why Choose the Earliest Date?*

Selecting the earliest termination date is the most reliable way to estimate when a business actually closed. Since directories do not continuously update terminated businesses, a later recorded date only confirms that the business was already closed by then—it does not indicate the exact moment of closure. By choosing the earliest available record, we minimize the risk of overestimating how long a business remained operational, making the analysis more accurate.

# Libraries

In [ ]:
#Google credentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

# Read Files

In [ ]:
path = "/content/drive/MyDrive/Mamak Stores/ACRA data/mamaks_b1980s_geo.csv"
mamaks_b1980s = pd.read_csv(path)

# 2.2. Webscraping with List of URLs (before 1980s)

The termination date of these non-live Sole Proprietory companies are listed on 2 well-known directories called sgpbusiness.com and recordowl.com. Each non-live company has a designated URL in both directories that leads to its profiles respectively. The webscraper (Octoparse) will search every link for the corresponding Termination Date of each non-live provision shop and collate them in a csv file.

In [ ]:
mm_b1980s_url = pd.DataFrame()
mm_b1980s_url['entity_name'] = mamaks_b1980s['entity_name']
mm_b1980s_url['formatted_name'] = mamaks_b1980s['entity_name'].str.lower()

In [ ]:
invalid_chars = ["'", "-", "(", ")", "."]
mm_b1980s_url['formatted_name'] = mm_b1980s_url['formatted_name'].str.replace(r"[\'\-\(\)]", '', regex=True)

In [ ]:
mm_b1980s_url['formatted_name'] = mm_b1980s_url['formatted_name'].str.replace('&', 'and', regex=True)
mm_b1980s_url['formatted_name'] = mm_b1980s_url['formatted_name'].str.replace(' ', '-', regex=True)
mm_b1980s_url['recordowl_url'] = 'https://recordowl.com/company/' + mm_b1980s_url['formatted_name']
mm_b1980s_url['sgbiz_url'] = 'https://sgpbusiness.com/company/' + mm_b1980s_url['formatted_name']

In [ ]:
mm_b1980s_url.head()

,entity_name,formatted_name,recordowl_url,sgbiz_url
0,WING THYE HUP KEE,wing-thye-hup-kee,https://recordowl.com/company/wing-thye-hup-kee,https://sgpbusiness.com/company/wing-thye-hup-kee
1,WING HENG & COMPANY,wing-heng-and-company,https://recordowl.com/company/wing-heng-and-co...,https://sgpbusiness.com/company/wing-heng-and-...
2,WIN & COMPANY,win-and-company,https://recordowl.com/company/win-and-company,https://sgpbusiness.com/company/win-and-company
3,WAN LEE,wan-lee,https://recordowl.com/company/wan-lee,https://sgpbusiness.com/company/wan-lee
4,W.S. RAM SINGH CO,w.s.-ram-singh-co,https://recordowl.com/company/w.s.-ram-singh-co,https://sgpbusiness.com/company/w.s.-ram-singh-co


### Save URLs

In [ ]:
recordowl_url_only_b = mm_b1980s_url['recordowl_url']
sgbiz_url_only_b = mm_b1980s_url['sgbiz_url']

In [ ]:
recordowl_url_only_b.head()

,recordowl_url
0,https://recordowl.com/company/wing-thye-hup-kee
1,https://recordowl.com/company/wing-heng-and-co...
2,https://recordowl.com/company/win-and-company
3,https://recordowl.com/company/wan-lee
4,https://recordowl.com/company/w.s.-ram-singh-co


In [ ]:
sgbiz_url_only_b.head()

,sgbiz_url
0,https://sgpbusiness.com/company/wing-thye-hup-kee
1,https://sgpbusiness.com/company/wing-heng-and-...
2,https://sgpbusiness.com/company/win-and-company
3,https://sgpbusiness.com/company/wan-lee
4,https://sgpbusiness.com/company/w.s.-ram-singh-co


In [ ]:
#SAVE
recordowl_url_only_b.to_csv('/content/drive/MyDrive/Mamak Stores/shop_termination_data/recordowl_url_only_b.csv', index=False)
sgbiz_url_only_b.to_csv('/content/drive/MyDrive/Mamak Stores/shop_termination_data/sgbiz_url_only_b.csv', index=False)

# 3. Cleaning SGBusiness Data from Webscraping

In [ ]:
file_path_1 = '/content/drive/MyDrive/Mamak Stores/shop_termination_data/nlive_termination_data_b.csv'
termination_data = pd.read_csv(file_path_1)

In [ ]:
termination_data.head()

,entity_name,termination_date
0,WING THYE HUP KEE,Terminated as on 16 July 2020 (Thursday)
1,WING HENG & COMPANY,Terminated as on 16 July 2020 (Thursday)
2,WIN & COMPANY,Terminated as on 16 July 2020 (Thursday)
3,WAN LEE,Terminated as on 16 July 2020 (Thursday)
4,NaN,NaN


In [ ]:
day_list = ['(Monday)', '(Tuesday)', '(Wednesday)', '(Thursday)', '(Friday)', '(Saturday)', '(Sunday)']

for i in day_list:
  termination_data['termination_date'] = termination_data['termination_date'].str.replace(i, '')

termination_data['termination_date'] = termination_data['termination_date'].replace('()', '')
termination_data['termination_date'] = termination_data['termination_date'].replace('Terminated as on', '', regex=True)
termination_data['termination_date'] = termination_data['termination_date'].replace('Cancelled as on', '', regex=True)

In [ ]:
termination_data['termination_date'] = pd.to_datetime(
    termination_data['termination_date'], errors='coerce')

In [ ]:
termination_data.head()

,entity_name,termination_date
0,WING THYE HUP KEE,2020-07-16
1,WING HENG & COMPANY,2020-07-16
2,WIN & COMPANY,2020-07-16
3,WAN LEE,2020-07-16
4,NaN,NaT


In [ ]:
mamaks_b1980s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2317 entries, 0 to 2316
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                2317 non-null   object 
 1   issuance_agency_id                 2317 non-null   object 
 2   entity_name                        2317 non-null   object 
 3   entity_type_description            2317 non-null   object 
 4   business_constitution_description  2317 non-null   object 
 5   company_type_description           2317 non-null   object 
 6   paf_constitution_description       2317 non-null   object 
 7   entity_status_description          2317 non-null   object 
 8   registration_incorporation_date    2317 non-null   object 
 9   uen_issue_date                     2317 non-null   object 
 10  address_type                       2317 non-null   object 
 11  block                              2317 non-null   objec

In [ ]:
termination_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2317 entries, 0 to 2316
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   entity_name       2236 non-null   object        
 1   termination_date  1984 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 36.3+ KB


In [ ]:
termination_data['termination_year'] = termination_data['termination_date'].dt.year

In [ ]:
for i in range(mamaks_b1980s.shape[0] - 1):
    if mamaks_b1980s['entity_name'].iloc[i] == termination_data['entity_name'].iloc[i]:
        mamaks_b1980s.loc[i, 'termination_year_1'] = termination_data.loc[i, 'termination_year']
    else:
        mamaks_b1980s.loc[i + 1, 'termination_year_1'] = termination_data.loc[i, 'termination_year']

In [ ]:
mamaks_b1980s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2317 entries, 0 to 2316
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                2317 non-null   object 
 1   issuance_agency_id                 2317 non-null   object 
 2   entity_name                        2317 non-null   object 
 3   entity_type_description            2317 non-null   object 
 4   business_constitution_description  2317 non-null   object 
 5   company_type_description           2317 non-null   object 
 6   paf_constitution_description       2317 non-null   object 
 7   entity_status_description          2317 non-null   object 
 8   registration_incorporation_date    2317 non-null   object 
 9   uen_issue_date                     2317 non-null   object 
 10  address_type                       2317 non-null   object 
 11  block                              2317 non-null   objec

#4. Cleaning RecordOwl Data from Webscraping

In [ ]:
file_path_2 = '/content/drive/MyDrive/Mamak Stores/shop_termination_data/nlive_termination_data_owl_b.csv'
termination_data_owl = pd.read_csv(file_path_2)

In [ ]:
termination_data_owl.head()

,entity_name,entity_age
0,WING THYE HUP KEE (03056700M),"16 years, 7 months and 4 days"
1,NaN,NaN
2,NaN,NaN
3,WAN LEE (01404500A),"16 years, 7 months and 4 days"
4,NaN,NaN


In [ ]:
termination_data_owl['entity_name'] = termination_data_owl['entity_name'].str[:-11]

In [ ]:
import re

def extract_time_components(time_str):
    if not isinstance(time_str, str):  # Handle NaN or non-string values
        return (0, 0, 0)  # Default values for missing data

    pattern = r"(\d+)\s*years|(\d+)\s*months|(\d+)\s*days"
    matches = re.findall(pattern, time_str)

    years, months, days = 0, 0, 0  # Default values
    for match in matches:
        y, m, d = match  # Each match is a tuple of 3 elements
        if y: years = int(y)
        if m: months = int(m)
        if d: days = int(d)

    return years, months, days  # Return a tuple

In [ ]:
termination_data_owl['entity_name'] = termination_data_owl['entity_name'].fillna('')
termination_data_owl['time_components'] = termination_data_owl['entity_age'].apply(extract_time_components)

In [ ]:
termination_data_owl.head()

,entity_name,entity_age,time_components
0,WING THYE HUP KEE,"16 years, 7 months and 4 days","(16, 7, 4)"
1,,NaN,"(0, 0, 0)"
2,,NaN,"(0, 0, 0)"
3,WAN LEE,"16 years, 7 months and 4 days","(16, 7, 4)"
4,,NaN,"(0, 0, 0)"


In [ ]:
from datetime import datetime, timedelta
#Reference date (5th March 2025)
reference_date = datetime(2025, 3, 5)

In [ ]:
from dateutil.relativedelta import relativedelta

termination_data_owl['termination_date'] = termination_data_owl['time_components'].apply(
    lambda row: reference_date - relativedelta(years=row[0], months=row[1], days=row[2])
)

termination_data_owl['termination_year'] = termination_data_owl['termination_date'].dt.year

In [ ]:
termination_data_owl.head()

,entity_name,entity_age,time_components,termination_date,termination_year
0,WING THYE HUP KEE,"16 years, 7 months and 4 days","(16, 7, 4)",2008-08-01,2008
1,,NaN,"(0, 0, 0)",2025-03-05,2025
2,,NaN,"(0, 0, 0)",2025-03-05,2025
3,WAN LEE,"16 years, 7 months and 4 days","(16, 7, 4)",2008-08-01,2008
4,,NaN,"(0, 0, 0)",2025-03-05,2025


In [ ]:
for i in range(mamaks_b1980s.shape[0] - 1):
    if mamaks_b1980s['entity_name'].iloc[i] == termination_data_owl['entity_name'].iloc[i]:
        mamaks_b1980s.loc[i, 'termination_year_2'] = termination_data_owl.loc[i, 'termination_year']
    else:
        mamaks_b1980s.loc[i + 1, 'termination_year_2'] = termination_data_owl.loc[i, 'termination_year']

In [ ]:
mamaks_b1980s.head()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity,registration_year,Address,Add_lat,Add_lon,termination_year_1,termination_year_2
0,03056700M,ACRA,WING THYE HUP KEE,Business,Sole Proprietor,na,na,na,1974-12-09,2008-09-09,...,na,NaN,na,na,1974,127 KALLANG ROAD SINGAPORE 208706,1.308109,103.863335,2020.0,NaN
1,03091500K,ACRA,WING HENG & COMPANY,Business,Partnership,na,na,na,1974-11-26,2008-09-09,...,na,NaN,na,na,1974,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1.281754,103.844879,2020.0,2008.0
2,03143800L,ACRA,WIN & COMPANY,Business,Sole Proprietor,na,na,na,1974-12-07,2008-09-09,...,na,NaN,na,na,1974,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,1.311125,103.788211,2020.0,2025.0
3,03389400X,ACRA,WAN LEE,Business,Sole Proprietor,na,na,na,1974-12-11,2008-09-09,...,na,NaN,na,na,1974,128 ST. JOHN'S ROAD SINGAPORE 2775,1.456403,103.827852,2020.0,2025.0
4,03496500M,ACRA,W.S. RAM SINGH CO,Business,Sole Proprietor,na,na,na,1974-12-28,2008-09-09,...,na,NaN,na,na,1974,242 SERANGOON ROAD SINGAPORE 218087,1.309940,103.854199,NaN,2008.0


In [ ]:
#SAVE
mamaks_b1980s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_all_nlive_tdates_b.csv', index=False)

#5. Preparing Data From "Estimating Date of Termination of Non-live Shops by Proxy"

In [ ]:
file_path_3 = '/content/drive/MyDrive/Mamak Stores/shop_termination_data/termination_data_proxy_b.csv'
termination_data_proxy = pd.read_csv(file_path_3)

In [ ]:
termination_data_proxy.head()

,entity_name,Address,tdate_proxy,termination_year
0,WING THYE HUP KEE,127 KALLANG ROAD SINGAPORE 208706,1990-04-06,1990.0
1,WING HENG & COMPANY,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1992-01-28,1992.0
2,WIN & COMPANY,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,NaN,NaN
3,WAN LEE,128 ST. JOHN'S ROAD SINGAPORE 2775,NaN,NaN
4,W.S. RAM SINGH CO,242 SERANGOON ROAD SINGAPORE 218087,1975-04-04,1975.0


In [ ]:
for i in range(mamaks_b1980s.shape[0] - 1):
    if mamaks_b1980s['entity_name'].iloc[i] == termination_data_proxy['entity_name'].iloc[i]:
        mamaks_b1980s.loc[i, 'termination_year_3'] = termination_data_proxy.loc[i, 'termination_year']
    else:
        mamaks_b1980s.loc[i + 1, 'termination_year_3'] = termination_data_proxy.loc[i, 'termination_year']

In [ ]:
mamaks_b1980s.head()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity,registration_year,Address,Add_lat,Add_lon,termination_year_1,termination_year_2,termination_year_3
0,03056700M,ACRA,WING THYE HUP KEE,Business,Sole Proprietor,na,na,na,1974-12-09,2008-09-09,...,NaN,na,na,1974,127 KALLANG ROAD SINGAPORE 208706,1.308109,103.863335,2020.0,NaN,1990.0
1,03091500K,ACRA,WING HENG & COMPANY,Business,Partnership,na,na,na,1974-11-26,2008-09-09,...,NaN,na,na,1974,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1.281754,103.844879,2020.0,2008.0,1992.0
2,03143800L,ACRA,WIN & COMPANY,Business,Sole Proprietor,na,na,na,1974-12-07,2008-09-09,...,NaN,na,na,1974,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,1.311125,103.788211,2020.0,2025.0,NaN
3,03389400X,ACRA,WAN LEE,Business,Sole Proprietor,na,na,na,1974-12-11,2008-09-09,...,NaN,na,na,1974,128 ST. JOHN'S ROAD SINGAPORE 2775,1.456403,103.827852,2020.0,2025.0,NaN
4,03496500M,ACRA,W.S. RAM SINGH CO,Business,Sole Proprietor,na,na,na,1974-12-28,2008-09-09,...,NaN,na,na,1974,242 SERANGOON ROAD SINGAPORE 218087,1.309940,103.854199,NaN,2008.0,1975.0


#6. Get Approximate Termination Year of Non-live Shops

In [ ]:
mamaks_b1980s['termination_year'] = mamaks_b1980s[['termination_year_1', 'termination_year_2', 'termination_year_3']].min(axis=1)
mamaks_b1980s = mamaks_b1980s.drop(columns=['termination_year_1','termination_year_2','termination_year_3'])

In [ ]:
mamaks_b1980s.head()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity,registration_year,Address,Add_lat,Add_lon,termination_year
0,03056700M,ACRA,WING THYE HUP KEE,Business,Sole Proprietor,na,na,na,1974-12-09,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,127 KALLANG ROAD SINGAPORE 208706,1.308109,103.863335,1990.0
1,03091500K,ACRA,WING HENG & COMPANY,Business,Partnership,na,na,na,1974-11-26,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1.281754,103.844879,1992.0
2,03143800L,ACRA,WIN & COMPANY,Business,Sole Proprietor,na,na,na,1974-12-07,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,1.311125,103.788211,2020.0
3,03389400X,ACRA,WAN LEE,Business,Sole Proprietor,na,na,na,1974-12-11,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,128 ST. JOHN'S ROAD SINGAPORE 2775,1.456403,103.827852,2020.0
4,03496500M,ACRA,W.S. RAM SINGH CO,Business,Sole Proprietor,na,na,na,1974-12-28,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,242 SERANGOON ROAD SINGAPORE 218087,1.309940,103.854199,1975.0


In [ ]:
#SAVE
mamaks_b1980s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_all_nlive_tdates_b.csv', index=False)

#7. Separating Data

This part of the analysis will aid the mapping of mamak shops from the 1980s until today. This is to understand how the numbers change and if there are spatial elements that contribute to these changes. It is also imperative to visualise the growth and de-growth of mamak shops across the decades for others to read and understand the situation.

* mm_live_xxxx: shops that are live until today
* mm_live_atm_xxxx: what shops were live during the decade but are no longer live

In [ ]:
def live_shops_atm(start_yr, end_yr):

    #shops registered within the period
    shops_tdec = mamaks_b1980s[
        (mamaks_b1980s['registration_year'] >= start_yr) &
        (mamaks_b1980s['registration_year'] <= end_yr)
    ]

    #shops registered before the period but still open within the period
    shops_pre_tdec_open_tdec = mamaks_b1980s[
        (mamaks_b1980s['registration_year'] < start_yr) &
        (mamaks_b1980s['termination_year'] > end_yr)
    ]

    #combine and remove duplicates based on entity_name
    result = pd.concat([shops_tdec, shops_pre_tdec_open_tdec]).drop_duplicates(subset=['entity_name']).reset_index(drop=True)

    return result


In [ ]:
mm_live_atm_1980s = live_shops_atm(1980, 1989)
mm_live_atm_1980s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                1539 non-null   object 
 1   issuance_agency_id                 1539 non-null   object 
 2   entity_name                        1539 non-null   object 
 3   entity_type_description            1539 non-null   object 
 4   business_constitution_description  1539 non-null   object 
 5   company_type_description           1539 non-null   object 
 6   paf_constitution_description       1539 non-null   object 
 7   entity_status_description          1539 non-null   object 
 8   registration_incorporation_date    1539 non-null   object 
 9   uen_issue_date                     1539 non-null   object 
 10  address_type                       1539 non-null   object 
 11  block                              1539 non-null   objec

In [ ]:
mm_live_atm_1990s = live_shops_atm(1990, 1999)
mm_live_atm_1990s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263 entries, 0 to 1262
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                1263 non-null   object 
 1   issuance_agency_id                 1263 non-null   object 
 2   entity_name                        1263 non-null   object 
 3   entity_type_description            1263 non-null   object 
 4   business_constitution_description  1263 non-null   object 
 5   company_type_description           1263 non-null   object 
 6   paf_constitution_description       1263 non-null   object 
 7   entity_status_description          1263 non-null   object 
 8   registration_incorporation_date    1263 non-null   object 
 9   uen_issue_date                     1263 non-null   object 
 10  address_type                       1263 non-null   object 
 11  block                              1263 non-null   objec

In [ ]:
mm_live_atm_2000s = live_shops_atm(2000, 2009)
mm_live_atm_2000s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                115 non-null    object 
 1   issuance_agency_id                 115 non-null    object 
 2   entity_name                        115 non-null    object 
 3   entity_type_description            115 non-null    object 
 4   business_constitution_description  115 non-null    object 
 5   company_type_description           115 non-null    object 
 6   paf_constitution_description       115 non-null    object 
 7   entity_status_description          115 non-null    object 
 8   registration_incorporation_date    115 non-null    object 
 9   uen_issue_date                     115 non-null    object 
 10  address_type                       115 non-null    object 
 11  block                              115 non-null    object 

In [ ]:
mm_live_atm_2010s = live_shops_atm(2010, 2019)
mm_live_atm_2010s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                43 non-null     object 
 1   issuance_agency_id                 43 non-null     object 
 2   entity_name                        43 non-null     object 
 3   entity_type_description            43 non-null     object 
 4   business_constitution_description  43 non-null     object 
 5   company_type_description           43 non-null     object 
 6   paf_constitution_description       43 non-null     object 
 7   entity_status_description          43 non-null     object 
 8   registration_incorporation_date    43 non-null     object 
 9   uen_issue_date                     43 non-null     object 
 10  address_type                       43 non-null     object 
 11  block                              43 non-null     object 
 

In [ ]:
mm_live_atm_2020s = live_shops_atm(2020,2025)
mm_live_atm_2020s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                0 non-null      object 
 1   issuance_agency_id                 0 non-null      object 
 2   entity_name                        0 non-null      object 
 3   entity_type_description            0 non-null      object 
 4   business_constitution_description  0 non-null      object 
 5   company_type_description           0 non-null      object 
 6   paf_constitution_description       0 non-null      object 
 7   entity_status_description          0 non-null      object 
 8   registration_incorporation_date    0 non-null      object 
 9   uen_issue_date                     0 non-null      object 
 10  address_type                       0 non-null      object 
 11  block                              0 non-null      object 
 12  street

In [ ]:
#SAVE
mm_live_atm_1990s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_live_atm_1980s_b.csv', index=False)
mm_live_atm_1990s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_live_atm_1990s_b.csv', index=False)
mm_live_atm_2000s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_live_atm_2000s_b.csv', index=False)
mm_live_atm_2010s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_live_atm_2010s_b.csv', index=False)
mm_live_atm_2020s.to_csv('/content/drive/MyDrive/Mamak Stores/qgis/live_atm/mm_live_atm_2020s_b.csv', index=False)

There were about 2000 of these provision shops that were registered since the 1980s that were opened between the 1990s to the early 2000s. The 90s had the highest number of live stores, followed by the early 2000s. However, there was a sharp drop after that, leaving the 2010s with only about 59% of that number. The data visualisation part of this project will provide a better perspective of the scope.